Journal of Civil Engineering and Management - 4887

<!-- https://www.tandfonline.com/action/doSearch?AllField=BIM+and+Green+Building&SeriesKey=tcem20 -->



In [3]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd

In [4]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df


,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
# searchTerm1 = ['BIM', 'Building Information Modelling']
# searchTerm2 = ['Green Building', 'Green Buildings']

tc = 0
searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

# print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

# print(programSearchTerms1)


# https://www.tandfonline.com/action/doSearch?AllField=BIM+and+Green+Building&SeriesKey=rcme20
    
baseUrl = 'https://www.tandfonline.com/action/doSearch?AllField='
querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        
        query = baseUrl + item1 +"+AND+"+ item2+"&SeriesKey=tcem20"
#         print(query)

        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')
        allParas = soup.find_all("p")
        dd = allParas[0].text
        miniData = dd.split(' ')
        NumberOfResultsStr = miniData[2]
        indexOfcomma = NumberOfResultsStr.find(',')

        if indexOfcomma != -1:
            NumberOfResultsStr = NumberOfResultsStr.replace(',','')

        number = int(NumberOfResultsStr)
        print(tc)
        tc = tc + number

        page = number // 10
        lastPage = number % 10
        if lastPage > 0:
            page = page + 1

        pageSize = page
        print(pageSize)

        cnt = 0
        allLinks = []
        
#         for testing purpose only
#         pageSize =1 
        for currentPage in range(0,pageSize):
            print("In page: ",currentPage)
            
            if currentPage != 0:
                newQuery = query + '&pageSize=10&subjectTitle=&startPage=' + str(currentPage)
                cnt = cnt + 1
            else:
                newQuery = query
            
            print("Searching for query: ",newQuery)
            response = requests.get(newQuery)
            soup = BeautifulSoup(response.text, 'html.parser')
            mydivs = soup.findAll("a", {'class':['ref nowrap']})
            
            for d in mydivs:
                url = 'https://www.tandfonline.com' + d.get('href')
                allLinks.append(url)

        for link in allLinks:
            df.loc[row,'searchTerm1'] = item1
            df.loc[row,'searchTerm2'] = item2
            df.loc[row,'url'] = link
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, "lxml")
            
            for i in soup.find('span',{'class':['NLM_article-title' ,'hlFld-title']}):
#                 print("Title: ",i)
                df.loc[row,'Title'] = i
            
            temp = soup.findAll('h2',{'class':['section-heading-2']} )
#             print("Number of sections: ",len(temp))            
            
            count = 0
            for i in range(len(temp)):

                if temp[i].text == temp[i].find_next('p').text:
        
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)                    
                    df.loc[row,sectionData] = extractData
                    count = count + 1
                else:
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)
                    df.loc[row,sectionData] = extractData
                    count = count + 1
            
            
            row = row + 1
        print("Row no: ",row)


0
3
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=tcem20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=2
Row no:  21
21
3
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Buildings&SeriesKey=tcem20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Buildings&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Buildings&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=2
Row no:  42
42
3
In page:  0
Searching for query:  https://www

In page:  7
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Green+Buildings&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=7
Row no:  387
387
19
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Sustainable+building&SeriesKey=tcem20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Sustainable+building&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Sustainable+building&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=2
In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Sustainable+building&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=3
In page:  4
Searching for query: 

In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Environmentally+friendly+design&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=3
In page:  4
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Environmentally+friendly+design&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=4
In page:  5
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Environmentally+friendly+design&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=5
In page:  6
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Environmentally+friendly+design&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=6
Row no:  813
813
19
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Sustainability&SeriesK

In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Energy&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Energy&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=2
In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Energy&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=3
In page:  4
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Energy&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=4
In page:  5
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Energy&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=5
In page:  6
Searching for query:  https://www.tandfonline.com/action/doSearch?Al

In page:  4
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Waste&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=4
In page:  5
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Waste&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=5
In page:  6
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Waste&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=6
In page:  7
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Waste&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=7
In page:  8
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+Information+Modelling+AND+Waste&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=8
In page:  9
Searching for query:  https://www.tandfonline.com/action/doSearch?AllFiel

...

In page:  10
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Water&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=10
In page:  11
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Water&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=11
In page:  12
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Water&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=12
In page:  13
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Water&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=13
In page:  14
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Water&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=14
In page:  15
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Buildin

In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Life+cycle+assessment&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=2
In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Life+cycle+assessment&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=3
In page:  4
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Life+cycle+assessment&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=4
In page:  5
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Life+cycle+assessment&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=5
In page:  6
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Building+information+model+AND+Life+cycle+assessment&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=6
In page:  7
Searching for

In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+construction+AND+Sustainable+development&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=2
Row no:  4022
4022
6
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+construction+AND+Energy&SeriesKey=tcem20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+construction+AND+Energy&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+construction+AND+Energy&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=2
In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+construction+AND+Energy&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=3
In page:  4
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+construction+AND+Energy&SeriesKey=t

In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Smart+construction+AND+Waste&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=1
Row no:  4456
4456
1
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Smart+construction+AND+Thermal+comfort&SeriesKey=tcem20
Row no:  4462
4462
2
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Smart+construction+AND+Electricity&SeriesKey=tcem20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Smart+construction+AND+Electricity&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=1
Row no:  4473
4473
1
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Smart+construction+AND+Solar&SeriesKey=tcem20
Row no:  4482
4482
1
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Smart+construction+AND+Wind+power&SeriesKey=tcem20
Row no:  4489
4

4828
1
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Solar&SeriesKey=tcem20
Row no:  4838
4838
1
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Wind+power&SeriesKey=tcem20
Row no:  4848
4848
1
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Green+infrastructure&SeriesKey=tcem20
Row no:  4857
4857
3
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Life+cycle+assessment&SeriesKey=tcem20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Life+cycle+assessment&SeriesKey=tcem20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Life+cycle+assessment&SeriesKey=tcem20&pageSize=10&subjectT

In [6]:
df.iloc[:, 0:36]

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section11,sectionData11,section12,sectionData12,section13,sectionData13,section14,sectionData14,section15,sectionData15
0,BIM,Green+Building,https://www.tandfonline.com/doi/abs/10.3846/13...,A model for spatial planning of site and build...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BIM,Green+Building,https://www.tandfonline.com/doi/abs/10.3846/13...,A process to divide curved walls in IFC-BIM in...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BIM,Green+Building,https://www.tandfonline.com/doi/abs/10.3846/13...,Developing as-built BIM model process manageme...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Green+Building,https://www.tandfonline.com/doi/abs/10.3846/13...,Evolution in the intellectual structure of BIM...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BIM,Green+Building,https://www.tandfonline.com/doi/abs/10.3846/13...,Construction 3D BIM-based knowledge management...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/abs/10.3846/13...,The latent causes of rework in floating produc...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4883,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/abs/10.3846/13...,"Analysis of problems, consequences and solutio...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4884,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/abs/10.3846/13...,A Multiple criteria decision support system fo...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4885,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/abs/10.1080/13...,EFFICIENCY INCREASE IN RESEARCH AND STUDIES WH...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
df = df.where(pd.notnull(df), None)


NameError: name 'df' is not defined

In [8]:
pwd

'/Users/chirag'

In [9]:
cd Desktop/MDSOutput

/Users/chirag/Desktop/MDSoutput


In [10]:
df.to_csv('KG-JCEM-4887.csv')